# Azure AI Agent with Existing Agent Example

This notebook demonstrates working with pre-existing Azure AI Agents by providing agent IDs, showing agent reuse patterns for production scenarios.

## Features Covered:
- Creating and managing persistent Azure AI agents
- Working with existing agent IDs
- Agent lifecycle management (create, use, delete)
- Production patterns for agent reuse
- Manual cleanup of agents

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

This example demonstrates how to work with an existing Azure AI agent by connecting to it using its agent ID.

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [9]:
import os
from pathlib import Path
from random import randint
from typing import Annotated
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field


## Initial Setup

Initialize the Azure AI project client and set up authentication.

In [ ]:
# This cell is for demonstration purposes only - actual async client setup is in the main() function
# The main() function uses the proper async imports from cell 4
from pathlib import Path  # For working with file paths
import os  # For environment variables
import time  # For sleep function
from dotenv import load_dotenv  # For loading environment variables from .env file

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('.env')  # Load environment variables from .env file

print("📝 Note: The actual client initialization happens in the main() function using async context managers")
print("✅ Environment variables and async imports are ready for use")

📝 Note: The actual client initialization happens in the main() function using async context managers
✅ Environment variables and async imports are ready for use


## Check Environment Variables

Let's verify that the required environment variables are set:

In [21]:
# Check required environment variables
required_vars = ["AZURE_AI_PROJECT_ENDPOINT", "AZURE_AI_MODEL_DEPLOYMENT_NAME"]
missing_vars = []

for var in required_vars:
    value = os.getenv(var)
    if value:
        print(f"✅ {var}: {value[:50]}..." if len(value) > 50 else f"✅ {var}: {value}")
    else:
        print(f"❌ {var}: Not set")
        missing_vars.append(var)

if missing_vars:
    print(f"\n⚠️  Please set the following environment variables: {', '.join(missing_vars)}")
else:
    print("\n✅ All required environment variables are set!")

✅ AZURE_AI_PROJECT_ENDPOINT: https://kd-foundry-project-resource.services.ai.az...
✅ AZURE_AI_MODEL_DEPLOYMENT_NAME: gpt-4o

✅ All required environment variables are set!


## Define Function Tools

Let's define a simple weather function that our agent can use:

In [22]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

## Create and Use Existing Agent

This example shows how to:
1. Create an agent that persists beyond the current session
2. Use the agent ID to work with an existing agent
3. Properly clean up resources

In [26]:
async def main() -> None:
    print("=== Azure AI Chat Client with Existing Agent ===")

    # Create the client using async versions
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        # Create an agent that will persist
        created_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"], 
            name="WeatherAgent"
        )
        
        print(f"✅ Created agent with ID: {created_agent.id}")

        try:
            async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=AzureAIAgentClient(project_client=client, agent_id=created_agent.id),
                instructions="You are a helpful weather agent.",
                tools=get_weather,
            ) as agent:
                result = await agent.run("What's the weather like in Tokyo?")
                print(f"Result: {result}\n")
        finally:
            # Clean up the agent manually (uncomment the lines below if you want to delete the agent)
            # await client.agents.delete_agent(created_agent.id)
            # print(f"🗑️ Deleted agent with ID: {created_agent.id}")
            print(f"💡 Agent with ID {created_agent.id} is still available for reuse")

## Execute the Example

Run the main function to see the existing agent workflow:

In [27]:
# Run the main function
await main()

=== Azure AI Chat Client with Existing Agent ===
✅ Created agent with ID: asst_2XKr7UoD1qZigI8m5oEfFA3Z
✅ Created agent with ID: asst_2XKr7UoD1qZigI8m5oEfFA3Z
Result: The weather in Tokyo is sunny with a high of 23°C.

💡 Agent with ID asst_2XKr7UoD1qZigI8m5oEfFA3Z is still available for reuse
Result: The weather in Tokyo is sunny with a high of 23°C.

💡 Agent with ID asst_2XKr7UoD1qZigI8m5oEfFA3Z is still available for reuse


## Working with Agent from Portal

Here's an example of how to work with an agent that was created through the Azure AI Studio portal:

In [28]:
async def use_portal_agent_example():
    """Example showing how to use an agent created in the Azure AI Studio portal."""
    print("=== Using Agent from Portal Example ===")
    
    # If you have an agent ID from the portal, you can use it directly
    # Replace 'your-agent-id-here' with the actual agent ID from the portal
    portal_agent_id = "asst_2XKr7UoD1qZigI8m5oEfFA3Z"  # Get this from Azure AI Studio
    
    print(f"Note: To use a portal agent, replace 'your-agent-id-here' with your actual agent ID")
    print(f"Current example agent ID: {portal_agent_id}")
    
    # Uncomment and modify the following code to use a real portal agent:
    """
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(
                async_credential=credential,
                agent_id=portal_agent_id  # Use your actual agent ID here
            ),
            tools=get_weather,  # Add any additional tools needed
        ) as agent,
    ):
        result = await agent.run("What's the weather like in London?")
        print(f"Portal Agent Result: {result}")
    """

# Run the portal agent example
await use_portal_agent_example()

=== Using Agent from Portal Example ===
Note: To use a portal agent, replace 'your-agent-id-here' with your actual agent ID
Current example agent ID: asst_2XKr7UoD1qZigI8m5oEfFA3Z


## Advanced Example: Persistent Agent Management

This example shows how to create, list, and manage multiple agents:

In [30]:
async def agent_management_example():
    """Advanced example showing agent management operations."""
    print("=== Agent Management Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        # List existing agents
        print("📋 Existing agents:")
        agents = client.agents.list_agents()
        async for agent in agents:
            print(f"  - {agent.name} (ID: {agent.id})")
        
        # Create a new persistent agent
        new_agent = await client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
            name="PersistentWeatherAgent",
            instructions="You are a persistent weather agent that provides detailed weather information."
        )
        print(f"\n✅ Created persistent agent: {new_agent.name} (ID: {new_agent.id})")
        
        try:
            # Use the agent multiple times
            async with ChatAgent(
                chat_client=AzureAIAgentClient(project_client=client, agent_id=new_agent.id),
                tools=get_weather,
            ) as agent:
                
                queries = [
                    "What's the weather in New York?",
                    "How about in Paris?",
                    "Compare the weather in Tokyo and Sydney"
                ]
                
                for query in queries:
                    print(f"\n🤔 User: {query}")
                    result = await agent.run(query)
                    print(f"🤖 Agent: {result.text}")
                    
        finally:
            # Clean up the agent manually (uncomment the lines below if you want to delete the agent)
            # await client.agents.delete_agent(created_agent.id)
            # print(f"🗑️ Deleted agent with ID: {created_agent.id}")
            print(f"💡 Agent with ID {new_agent.id} is still available for reuse")

# Run the agent management example
await agent_management_example()

=== Agent Management Example ===
📋 Existing agents:
  - PersistentWeatherAgent (ID: asst_Uw0banRJdc0vIfa9XDNscOWG)
  - WeatherAgent (ID: asst_2XKr7UoD1qZigI8m5oEfFA3Z)
  - Agent454 (ID: asst_aEmCAyB7Byvg2xXOzAiyeyJL)
  - triage-agent (ID: asst_n2CVbVsYFFk8TO7WyGIWyYa2)
  - effort_agent (ID: asst_gVCB5mrQHHwI0Q6VGUCXkdXD)
  - team_agent (ID: asst_0jH94jKRHdUMZJdm2YiNi1Yb)
  - priority_agent (ID: asst_CBbi1JUrQLfaQmBuYFRFCrVS)
  - triage-agent (ID: asst_LMRBMphI7Bp24S5pV9RHc5Ge)
  - effort_agent (ID: asst_ciKwLHxru1PpRcTmt57RITza)
  - team_agent (ID: asst_JdP9ibnuJrLRyodn4Vu5kVgj)
  - priority_agent (ID: asst_VEv4cFlimau3ECIG0HNv1E2W)
  - fitness-search-agent (ID: asst_lc7GNNOYv88HeFlfBy6pWXJg)
  - health-bing-agent (ID: asst_M7VY9fo3OtFjxz1s3Oosx3Vs)
  - health-bing-agent (ID: asst_6vBSIm0AzWu4EK9RY7u3smmG)
  - health-search-agent (ID: asst_unyGnRHRU8Av1dWNSt7WGgNf)
  - health-calculator-agent (ID: asst_DOxvz3s2WcxLOfspDblpRfEh)
  - fun-fit-health-advisor (ID: asst_rxzs677vglMfRc8Np1CAr

## Key Takeaways

1. **Persistent Agents**: Agents can be created to persist beyond a single session, useful for production scenarios
2. **Agent ID Management**: You can work with agents using their IDs, whether created programmatically or through the portal
3. **Resource Management**: Always clean up agents when they're no longer needed to avoid unnecessary costs
4. **Production Patterns**: This approach is suitable for production applications where agents need to be reused
5. **Portal Integration**: Agents created in Azure AI Studio can be used directly in code
6. **Client Flexibility**: You can create agents with or without providing a project client

## Best Practices

1. **Environment Variables**: Use environment variables for configuration to keep credentials secure
2. **Error Handling**: Always use try-finally blocks to ensure agent cleanup
3. **Agent Naming**: Use descriptive names for agents to make management easier
4. **Documentation**: Keep track of agent IDs and their purposes
5. **Resource Monitoring**: Monitor your Azure AI resource usage when working with persistent agents

## Use Cases

- **Production Applications**: Where agents need to persist across multiple user sessions
- **Team Collaboration**: Sharing agents across team members
- **Multi-Session Conversations**: Maintaining agent state across sessions
- **Agent Libraries**: Creating reusable agents for common tasks
- **Integration with Azure AI Studio**: Using agents created through the web interface